In [1]:
"""
Use this to run the train.py routine
"""

import datetime
import os
import time
import wandb
import presets
import torch
import torch.utils.data
import torchvision
import torchvision.models.detection
import torchvision.models.detection.mask_rcnn
import utils
from coco_utils import get_coco, get_coco_kp
from engine import evaluate, train_one_epoch
from group_by_aspect_ratio import create_aspect_ratio_groups, GroupedBatchSampler
from torchvision.transforms import InterpolationMode
from transforms import SimpleCopyPaste


print('is_available: ', torch.cuda.is_available())
print('device_count: ', torch.cuda.device_count())
print('current_device: ', torch.cuda.current_device())
print('current_device: ', torch.cuda.device(0))
print('get_device_name: ', torch.cuda.get_device_name(0))




is_available:  True
device_count:  1
current_device:  0
current_device:  <torch.cuda.device object at 0x000002190269D940>
get_device_name:  NVIDIA GeForce RTX 3090


In [5]:


output_dir_path = r'C:\Users\endle\Desktop\pytorch-retinanet\outputdir'
data_dir_path = r"C:\Users\endle\Desktop\pytorch-retinanet\data"
dataset_type = 'coco'
model = "retinanet_resnet50_fpn"
device_type = "cuda"
batch_size = 8
epochs = 25
workers = 1
optimizer = "sgd"
norm_weight_decay = 0.9
momentum = 0.9
lr = 0.0005 #0.001
weight_decay = 1e-4
lr_step_size = 8

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="hair-disease-detector",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": model,
    "dataset": "coco",
    "epochs": epochs,
    }
)

def copypaste_collate_fn(batch):
    copypaste = SimpleCopyPaste(blending=True, resize_interpolation=InterpolationMode.BILINEAR)
    return copypaste(*utils.collate_fn(batch))


def get_dataset(name, image_set, transform, data_path):
    paths = {"coco": (data_path, get_coco, 91), "coco_kp": (data_path, get_coco_kp, 2)}
    p, ds_fn, num_classes = paths[name]

    ds = ds_fn(p, image_set=image_set, transforms=transform)
    return ds, num_classes


def get_transform(train, args):
    if train:
        return presets.DetectionPresetTrain(data_augmentation=args.data_augmentation)
    elif args.weights and args.test_only:
        weights = torchvision.models.get_weight(args.weights)
        trans = weights.transforms()
        return lambda img, target: (trans(img), target)
    else:
        return presets.DetectionPresetEval()


def get_args_parser(add_help=True):

    import argparse

    parser = argparse.ArgumentParser(description="PyTorch Detection Training", add_help=add_help)

    parser.add_argument("--data-path", default=data_dir_path, type=str, help="dataset path")
    parser.add_argument("--dataset", default=dataset_type, type=str, help="dataset name")
    parser.add_argument("--model", default=model, type=str, help="model name")
    parser.add_argument("--device", default=device_type, type=str, help="device (Use cuda or cpu Default: cuda)")
    parser.add_argument(
        "-b", "--batch-size", default=batch_size, type=int, help="images per gpu, the total batch size is $NGPU x batch_size"
    )
    parser.add_argument("--epochs", default=epochs, type=int, metavar="N", help="number of total epochs to run")
    parser.add_argument(
        "-j", "--workers", default=workers, type=int, metavar="N", help="number of data loading workers (default: 4)"
    )
    parser.add_argument("--opt", default=optimizer, type=str, help="optimizer")
    parser.add_argument(
        "--lr",
        default=lr,
        type=float,
        help="initial learning rate, 0.02 is the default value for training on 8 gpus and 2 images_per_gpu",
    )
    parser.add_argument("--momentum", default=momentum, type=float, metavar="M", help="momentum")
    parser.add_argument(
        "--wd",
        "--weight-decay",
        default=weight_decay,
        type=float,
        metavar="W",
        help="weight decay (default: 1e-4)",
        dest="weight_decay",
    )
    parser.add_argument(
        "--norm-weight-decay",
        default=norm_weight_decay,
        type=float,
        help="weight decay for Normalization layers (default: None, same value as --wd)",
    )
    parser.add_argument(
        "--lr-scheduler", default="multisteplr", type=str, help="name of lr scheduler (default: multisteplr)"
    )
    parser.add_argument(
        "--lr-step-size", default=lr_step_size, type=int, help="decrease lr every step-size epochs (multisteplr scheduler only)"
    )
    parser.add_argument(
        "--lr-steps",
        default=[16, 22],
        nargs="+",
        type=int,
        help="decrease lr every step-size epochs (multisteplr scheduler only)",
    )
    parser.add_argument(
        "--lr-gamma", default=0.1, type=float, help="decrease lr by a factor of lr-gamma (multisteplr scheduler only)"
    )
    parser.add_argument("--print-freq", default=20, type=int, help="print frequency")
    parser.add_argument("--output_dir", default=output_dir_path, type=str, help="path to save outputs")
    parser.add_argument("--resume", default=output_dir_path, type=str, help="path of checkpoint")
    parser.add_argument("--start_epoch", default=0, type=int, help="start epoch")
    parser.add_argument("--aspect-ratio-group-factor", default=3, type=int)
    parser.add_argument("--rpn-score-thresh", default=None, type=float, help="rpn score threshold for faster-rcnn")
    parser.add_argument(
        "--trainable-backbone-layers", default=None, type=int, help="number of trainable layers of backbone"
    )
    parser.add_argument(
        "--data-augmentation", default="hflip", type=str, help="data augmentation policy (default: hflip)"
    )
    parser.add_argument(
        "--sync-bn",
        dest="sync_bn",
        help="Use sync batch norm",
        action="store_true",
    )
    parser.add_argument(
        "--test-only",
        dest="test_only",
        help="Only test the model",
        action="store_true",
    )

    parser.add_argument(
        "--use-deterministic-algorithms", action="store_true", help="Forces the use of deterministic algorithms only."
    )

    # distributed training parameters
    parser.add_argument("--world-size", default=1, type=int, help="number of distributed processes")
    parser.add_argument("--dist-url", default="env://", type=str, help="url used to set up distributed training")
    parser.add_argument("--weights", default=None, type=str, help="the weights enum name to load")
    parser.add_argument("--weights-backbone", default='ResNet50_Weights.IMAGENET1K_V1', type=str, help="the backbone weights enum name to load")

    # Mixed precision training parameters
    parser.add_argument("--amp", default=True, action="store_true", help="Use torch.cuda.amp for mixed precision training")

    # Use CopyPaste augmentation training parameter
    parser.add_argument(
        "--use-copypaste",
        action="store_true",
        help="Use CopyPaste data augmentation. Works only with data-augmentation='lsj'.",
    )

    return parser


def main(args):

    #if args.output_dir:
    output_dir_path = r'C:\Users\endle\Desktop\pytorch-retinanet\outputdir'

    utils.mkdir(output_dir_path)

    utils.init_distributed_mode(args)
    print('what is in args: ,', args)

    device = torch.device(args.device)

    #if args.use_deterministic_algorithms:
    
    torch.use_deterministic_algorithms(True)

    # Data loading code
    print("Loading data")

    dataset, num_classes = get_dataset(args.dataset, "train", get_transform(True, args), args.data_path)
    dataset_test, _ = get_dataset(args.dataset, "val", get_transform(False, args), args.data_path)

    print("Creating data loaders")
    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test, shuffle=False)
    else:
        train_sampler = torch.utils.data.RandomSampler(dataset)
        test_sampler = torch.utils.data.SequentialSampler(dataset_test)

    if args.aspect_ratio_group_factor >= 0:
        group_ids = create_aspect_ratio_groups(dataset, k=args.aspect_ratio_group_factor)
        train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, args.batch_size)
    else:
        train_batch_sampler = torch.utils.data.BatchSampler(train_sampler, args.batch_size, drop_last=True)

    train_collate_fn = utils.collate_fn
    if args.use_copypaste:
        if args.data_augmentation != "lsj":
            raise RuntimeError("SimpleCopyPaste algorithm currently only supports the 'lsj' data augmentation policies")

        train_collate_fn = copypaste_collate_fn

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_sampler=train_batch_sampler, num_workers=args.workers, collate_fn=train_collate_fn
    )

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, sampler=test_sampler, num_workers=args.workers, collate_fn=utils.collate_fn
    )

    print("Creating model")
    kwargs = {"trainable_backbone_layers": args.trainable_backbone_layers}
    if args.data_augmentation in ["multiscale", "lsj"]:
        kwargs["_skip_resize"] = True
    if "rcnn" in args.model:
        if args.rpn_score_thresh is not None:
            kwargs["rpn_score_thresh"] = args.rpn_score_thresh
    # model = torchvision.models.get_model(
    #     args.model, weights=args.weights, weights_backbone=args.weights_backbone, num_classes=num_classes, **kwargs
    # )

    model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True, num_classes=num_classes, **kwargs)

    model.to(device)
    if args.distributed and args.sync_bn:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)

    model_without_ddp = model
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        model_without_ddp = model.module

    if args.norm_weight_decay is None:
        parameters = [p for p in model.parameters() if p.requires_grad]
    else:
        param_groups = torchvision.ops._utils.split_normalization_params(model)
        wd_groups = [args.norm_weight_decay, args.weight_decay]
        parameters = [{"params": p, "weight_decay": w} for p, w in zip(param_groups, wd_groups) if p]

    opt_name = args.opt.lower()
    if opt_name.startswith("sgd"):
        optimizer = torch.optim.SGD(
            parameters,
            lr=args.lr,
            momentum=args.momentum,
            weight_decay=args.weight_decay,
            nesterov="nesterov" in opt_name,
        )
    elif opt_name == "adamw":
        optimizer = torch.optim.AdamW(parameters, lr=args.lr, weight_decay=args.weight_decay)
    else:
        raise RuntimeError(f"Invalid optimizer {args.opt}. Only SGD and AdamW are supported.")

    scaler = torch.cuda.amp.GradScaler() if args.amp else None

    args.lr_scheduler = args.lr_scheduler.lower()
    if args.lr_scheduler == "multisteplr":
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=args.lr_steps, gamma=args.lr_gamma)
    elif args.lr_scheduler == "cosineannealinglr":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs)
    else:
        raise RuntimeError(
            f"Invalid lr scheduler '{args.lr_scheduler}'. Only MultiStepLR and CosineAnnealingLR are supported."
        )
    
    print('what is argsresume: ', args.resume)

    # if args.resume:
    #     checkpoint = torch.load(args.resume, map_location="cpu")
    #     model_without_ddp.load_state_dict(checkpoint["model"])
    #     optimizer.load_state_dict(checkpoint["optimizer"])
    #     lr_scheduler.load_state_dict(checkpoint["lr_scheduler"])
    #     args.start_epoch = checkpoint["epoch"] + 1
    #     if args.amp:
    #         scaler.load_state_dict(checkpoint["scaler"])

    if args.test_only:
        torch.backends.cudnn.deterministic = True
        evaluate(model, data_loader_test, device=device)
        return

    print("Start training")
    start_time = time.time()
    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)

        #print('look at data_loader: ', data_loader) # <torch.utils.data.dataloader.DataLoader object at 0x00000188DE551940>
        #print('len of dataloader: ', len(data_loader))

        x = train_one_epoch(model, optimizer, data_loader, device, epoch, args.print_freq, scaler)
        
        print('metric logger avg stats: ', x) # metric logger avg stats:  lr: 0.000500  loss: 1.1949 (1.6263)  classification: 0.6939 (1.0290)  bbox_regression: 0.4993 (0.5973)

        print(x.meters.get('loss')) # 0.9397 (0.9770)

        lr_scheduler.step()
        if args.output_dir:
            checkpoint = {
                "model": model_without_ddp.state_dict(),
                "optimizer": optimizer.state_dict(),
                "lr_scheduler": lr_scheduler.state_dict(),
                "args": args,
                "epoch": epoch,
            }
            if args.amp:
                checkpoint["scaler"] = scaler.state_dict()
            utils.save_on_master(checkpoint, os.path.join(args.output_dir, f"model_{epoch}.pth"))
            utils.save_on_master(checkpoint, os.path.join(args.output_dir, "checkpoint.pth"))

        # evaluate after every epoch
        c = evaluate(model, data_loader_test, device=device)

        for iou_type, coco_eval in c.coco_eval.items():
            wandb.log({"AP/IoU/0.50-0.95/all/100": coco_eval.stats[0], 'epoch' : epoch})
            wandb.log({"AP/IoU/0.50/all/100": coco_eval.stats[1], 'epoch' : epoch})
            wandb.log({"AP/IoU/0.75/all/100": coco_eval.stats[2], 'epoch' : epoch})
            wandb.log({"AP/IoU/0.50-0.95/small/100": coco_eval.stats[3], 'epoch' : epoch})
            wandb.log({"AP/IoU/0.50-0.95/medium/100": coco_eval.stats[4], 'epoch' : epoch})
            wandb.log({"AP/IoU/0.50-0.95/large/100": coco_eval.stats[5], 'epoch' : epoch})
            wandb.log({"AR/IoU/0.50-0.95/all/1": coco_eval.stats[6], 'epoch' : epoch})
            wandb.log({"AR/IoU/0.50-0.95/all/10": coco_eval.stats[7], 'epoch' : epoch})
            wandb.log({"AR/IoU/0.50-0.95/all/100": coco_eval.stats[8], 'epoch' : epoch})
            wandb.log({"AR/IoU/0.50-0.95/small/100": coco_eval.stats[9], 'epoch' : epoch})
            wandb.log({"AR/IoU/0.50-0.95/medium/100": coco_eval.stats[10], 'epoch' : epoch})
            wandb.log({"AR/IoU/0.50-0.95/large/100": coco_eval.stats[11], 'epoch' : epoch})

        # for iou_type, coco_eval in c.coco_eval.items():
        #     print(f"IoU metric: {iou_type}")

        #     print('summary stuff: ')
        #     print(coco_eval.values())


    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print(f"Training time {total_time_str}")

AP/IoU/0.50-0.95/all/100,▁▅█
AP/IoU/0.50-0.95/large/100,▁▅█
AP/IoU/0.50-0.95/medium/100,▁▃█
AP/IoU/0.50-0.95/small/100,▁▁▁
AP/IoU/0.50/all/100,▁▆█
AP/IoU/0.75/all/100,▁▄█
AR/IoU/0.50-0.95/all/1,▁▆█
AR/IoU/0.50-0.95/all/10,▁▆█
AR/IoU/0.50-0.95/all/100,▁▆█
AR/IoU/0.50-0.95/large/100,▁▆█
AR/IoU/0.50-0.95/medium/100,▁▅█


In [6]:
# run it

#'--nproc_per_node = 0 --data-path = C:\Users\endle\Desktop\pytorch-retinanet\data
# --dataset coco 
# --model retinanet_resnet50_fpn 
# --epochs 26 
# --lr-steps 16 22 
# --aspect-ratio-group-factor 3 
# --lr 0.001 
# --weights-backbone ResNet50_Weights.IMAGENET1K_V1 train.py\

#ya = ['--nproc_per_node', '1']
yo = ['--weights-backbone', 'ResNet50_Weights.IMAGENET1K_V1']
args = get_args_parser().parse_args(yo)

#args.add_argument("--device", default="cuda")
print(args)
main(args)

Namespace(amp=True, aspect_ratio_group_factor=3, batch_size=8, data_augmentation='hflip', data_path='C:\\Users\\endle\\Desktop\\pytorch-retinanet\\data', dataset='coco', device='cuda', dist_url='env://', epochs=25, lr=0.0005, lr_gamma=0.1, lr_scheduler='multisteplr', lr_step_size=8, lr_steps=[16, 22], model='retinanet_resnet50_fpn', momentum=0.9, norm_weight_decay=0.9, opt='sgd', output_dir='C:\\Users\\endle\\Desktop\\pytorch-retinanet\\outputdir', print_freq=20, resume='C:\\Users\\endle\\Desktop\\pytorch-retinanet\\outputdir', rpn_score_thresh=None, start_epoch=0, sync_bn=False, test_only=False, trainable_backbone_layers=None, use_copypaste=False, use_deterministic_algorithms=False, weight_decay=0.0001, weights=None, weights_backbone='ResNet50_Weights.IMAGENET1K_V1', workers=1, world_size=1)
Not using distributed mode
what is in args: , Namespace(amp=True, aspect_ratio_group_factor=3, batch_size=8, data_augmentation='hflip', data_path='C:\\Users\\endle\\Desktop\\pytorch-retinanet\\dat

c:\Users\endle\miniconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\endle\miniconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


what is argsresume:  C:\Users\endle\Desktop\pytorch-retinanet\outputdir
Start training
Epoch: [0]  [ 0/46]  eta: 0:02:37  lr: 0.000012  loss: 2.6224 (2.6224)  classification: 1.8558 (1.8558)  bbox_regression: 0.7666 (0.7666)  time: 3.4276  data: 2.5219  max mem: 11200
Epoch: [0]  [20/46]  eta: 0:00:22  lr: 0.000234  loss: 1.9867 (2.0618)  classification: 1.3347 (1.3741)  bbox_regression: 0.6740 (0.6877)  time: 0.7173  data: 0.0068  max mem: 11439
Epoch: [0]  [40/46]  eta: 0:00:04  lr: 0.000456  loss: 1.2586 (1.6934)  classification: 0.7274 (1.0742)  bbox_regression: 0.5313 (0.6192)  time: 0.6836  data: 0.0070  max mem: 11447
Epoch: [0]  [45/46]  eta: 0:00:00  lr: 0.000500  loss: 1.1858 (1.6312)  classification: 0.6733 (1.0276)  bbox_regression: 0.5137 (0.6036)  time: 0.6897  data: 0.0070  max mem: 11447
Epoch: [0] Total time: 0:00:35 (0.7649 s / it)
metric logger avg stats:  lr: 0.000500  loss: 1.1858 (1.6312)  classification: 0.6733 (1.0276)  bbox_regression: 0.5137 (0.6036)
1.1858 (1

In [4]:
# save the model

#mdl = torch.save(checkpoint, os.path.join(args.output_dir, "checkpoint.pth"))

# load it
yo = ['--weights-backbone', 'ResNet50_Weights.IMAGENET1K_V1']
args = get_args_parser().parse_args(yo)


print("Creating model")
kwargs = {"trainable_backbone_layers": args.trainable_backbone_layers}
if args.data_augmentation in ["multiscale", "lsj"]:
    kwargs["_skip_resize"] = True
if "rcnn" in args.model:
    if args.rpn_score_thresh is not None:
        kwargs["rpn_score_thresh"] = args.rpn_score_thresh
        
output_dir_path = r'C:\Users\endle\Desktop\pytorch-retinanet\outputdir'


model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True, num_classes=91, **kwargs)

model.load_state_dict(torch.load(os.path.join(output_dir_path, "checkpoint.pth")))

model.eval()

Creating model


RuntimeError: Error(s) in loading state_dict for RetinaNet:
	Missing key(s) in state_dict: "backbone.body.conv1.weight", "backbone.body.bn1.weight", "backbone.body.bn1.bias", "backbone.body.bn1.running_mean", "backbone.body.bn1.running_var", "backbone.body.layer1.0.conv1.weight", "backbone.body.layer1.0.bn1.weight", "backbone.body.layer1.0.bn1.bias", "backbone.body.layer1.0.bn1.running_mean", "backbone.body.layer1.0.bn1.running_var", "backbone.body.layer1.0.conv2.weight", "backbone.body.layer1.0.bn2.weight", "backbone.body.layer1.0.bn2.bias", "backbone.body.layer1.0.bn2.running_mean", "backbone.body.layer1.0.bn2.running_var", "backbone.body.layer1.0.conv3.weight", "backbone.body.layer1.0.bn3.weight", "backbone.body.layer1.0.bn3.bias", "backbone.body.layer1.0.bn3.running_mean", "backbone.body.layer1.0.bn3.running_var", "backbone.body.layer1.0.downsample.0.weight", "backbone.body.layer1.0.downsample.1.weight", "backbone.body.layer1.0.downsample.1.bias", "backbone.body.layer1.0.downsample